In [1]:
import json
import numpy as np
from ase.lattice.cubic import FaceCenteredCubic, SimpleCubic, BodyCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.decahedron import Decahedron
from ase.cluster.octahedron import Octahedron
from debyecalculator import DebyeCalculator
import torch
import plotly.graph_objects as go

def load_dict(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)
    
def get_best_structure_details(result_simulated_Gr, num_best_fits=5, print_best=True):
    """
    Returns the best fits based on the rwp value.

    Parameters:
    result_simulated_Gr (dict): The results to sort and print.
    num_best_fits (int): The number of best fits to print. Default is 5.
    print_best (bool): Whether or not to print the best fits. Default is True.
    """
    # Flatten the results and sort them based on the rwp value
    flattened_results = [(k, v) for k, results in result_simulated_Gr.items() for v in results]
    sorted_results = sorted(flattened_results, key=lambda x: x[1]['rwp'])

    # Get the best total structures
    best_total_structures = sorted_results[:num_best_fits]

    if print_best:
        print("\nBest total structures across all types:")
        for structure_type, structure in best_total_structures:
            print(f"Structure Type: {structure_type}, Details: {structure}")
        print ()

    # Extract the structure_type, noshell, and lattice constant of the best structures
    best_structure_details = []
    for structure_type, structure in best_total_structures:
        details = {
            'structure_type': structure_type,
            'noshell': structure['noshell'],
            'lc': structure['lc']
        }
        best_structure_details.append(details)

    # Prepare the data for use with generate_and_evaluate_clusters
    structure_types = [details['structure_type'] for details in best_structure_details]
    lc_list = [details['lc'] for details in best_structure_details]
    noshells = [details['noshell'] for details in best_structure_details]

    return structure_types, lc_list, noshells

def LoadData(simulated_or_experimental='simulated', scatteringfunction='Gr'):
    # Set the filename based on the simulated_or_experimental and scatteringfunction variables
    if scatteringfunction == 'Gr':
        if simulated_or_experimental == 'simulated':
            filename = '../Data/Gr/Target_PDF_benchmark.npy'
        else:  # simulated_or_experimental == 'experimental'
            filename = '../Data/Gr/Experimental_PDF.gr'
    else:  # scatteringfunction == 'Sq'
        if simulated_or_experimental == 'simulated':
            filename = '../Data/Sq/Target_Sq_benchmark.npy'
        else:  # simulated_or_experimental == 'experimental'
            filename = '../Data/Sq/Experimental_Sq.sq'

    data = np.loadtxt(filename, skiprows=25) if filename.endswith('.gr') or filename.endswith('.sq') else np.load(filename)
    x_target = data[:, 0]
    Int_target = data[:, 1]

    return x_target, Int_target

def calculate_scattering(cluster, scatteringfunction='Gr'):
    """
    Calculate a Pair Distribution Function (PDF) or Structure Factor (Sq) from a given structure.

    Parameters:
    cluster (ase.Atoms): The atomic structure (from ASE Atoms object) to calculate the PDF or Sq from.
    scatteringfunction (str): The scatteringfunction to calculate. 'Gr' for pair distribution function, 'Sq' for structure factor. Default is 'Gr'.

    Returns:
    r/q (numpy.ndarray): The r values (for PDF) or q values (for Sq) from the calculated function.
    G/S (numpy.ndarray): The G values (for PDF) or S values (for Sq) from the calculated function.

    Raises:
    AssertionError: If the scatteringfunction parameter is not 'Gr' or 'Sq'.

    Example:
    >>> cluster = Icosahedron('Au', noshells=7)
    >>> r, G = calculate_scattering(cluster, scatteringfunction='Gr')
    >>> q, S = calculate_scattering(cluster, scatteringfunction='Sq')
    """
    # Check if the scatteringfunction parameter is valid
    assert scatteringfunction in ['Gr', 'Sq'], "scatteringfunction must be 'Gr' or 'Sq'"

    # Initialise calculator object
    calc = DebyeCalculator(qmin=2, qmax=10.0, rmax=30, qstep=0.01)

    # Extract atomic symbols and positions
    symbols = cluster.get_chemical_symbols()
    positions = cluster.get_positions()

    # Convert positions to a torch tensor
    positions_tensor = torch.tensor(positions)

    # Create a structure tuple
    structure_tuple = (symbols, positions_tensor)

    # Calculate Pair Distribution Function or Structure Factor
    if scatteringfunction == 'Gr':
        r, G = calc.gr(structure_source=structure_tuple)
        G /= G.max()
        return r, G
    else:  # scatteringfunction == 'Sq'
        q, S = calc.sq(structure_source=structure_tuple)
        S /= S.max()
        return q, S
    
def generate_and_evaluate_clusters(noshell, lc, structure_type, atom, surfaces, x_target, Int_target, scatteringfunction, plot=False):
    if structure_type == 'FaceCenteredCubic':
        cluster = FaceCenteredCubic(atom, directions=surfaces, size=[noshell]*3, latticeconstant=2*np.sqrt(0.5*lc**2))
    elif structure_type == 'SimpleCubic':
        cluster = SimpleCubic(atom, directions=surfaces, size=[noshell]*3, latticeconstant=lc)
    elif structure_type == 'BodyCenteredCubic':
        cluster = BodyCenteredCubic(atom, directions=surfaces, size=[noshell]*3, latticeconstant=lc)
    elif structure_type == 'HexagonalClosedPacked':
        cluster = HexagonalClosedPacked(atom, latticeconstant=(lc, lc*1.633), size=(noshell, noshell, noshell))
    elif structure_type == 'Icosahedron':
        cluster = Icosahedron(atom, noshell, 2*np.sqrt(0.5*lc**2))
    elif structure_type == 'Octahedron':
        cluster = Octahedron(atom, length=noshell, latticeconstant=2*np.sqrt(0.5*lc**2))
    elif structure_type == 'Decahedron':
        cluster = Decahedron(atom, noshell, noshell, noshell, 2*np.sqrt(0.5*lc**2))

    # Calculate the scattering pattern of the generated structure
    x_sim, Int_sim = calculate_scattering(cluster, scatteringfunction)

    # Interpolate the simulated intensity to the r/q values of the target scattering pattern
    Int_sim_interp = np.interp(x_target, x_sim, Int_sim)

    # Calculate the difference between the simulated and target scattering patterns
    diff = Int_target - Int_sim_interp

    # Calculate the Rwp value
    rwp = np.sqrt(np.sum(diff**2) / np.sum(Int_target**2))

    # If plot is True, generate an interactive plot of the target and simulated scattering patterns
    if plot:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_target, y=Int_target, mode='lines', name=f'Target {scatteringfunction}', line=dict(width=4)))  # Changed name based on scatteringfunction
        fig.add_trace(go.Scatter(x=x_target, y=Int_sim_interp, mode='lines', name=f'Simulated {scatteringfunction}'))  # Changed name based on scatteringfunction
        fig.update_layout(
            title=f"Structure Type: {structure_type}, No. of Shells: {noshell}, Lattice Constant: {lc:.2f}, Rwp Value: {rwp:.2f}",
            xaxis_title="r (Å)" if scatteringfunction == 'Gr' else "q (Å^-1)",
            yaxis_title="Intensity (a.u.)",
            legend_title="Legend"
        )
        fig.show()

    return rwp

# Results for simulated Gr data

In [2]:
# Load the results from the brute force search
result_simulated_Gr = load_dict('../results/bruteforce/result_simulated_Gr_fast_StructureSpace.json')

# Load the target PDF data
x_target, Int_target = LoadData(simulated_or_experimental='simulated', scatteringfunction='Gr')

# Get the best structure details
structure_types, lc_list, noshells = get_best_structure_details(result_simulated_Gr, num_best_fits=5, print_best=True)
atom = 'Au'
surfaces = [[1, 0, 0], [1, 1, 0], [1, 1, 1]] # Define the surfaces of the cluster
scatteringfunction = 'Gr'

# Loop through the best structures and calculate the Rwp value for each one
for i, (structure_type, lc, noshell) in enumerate(zip(structure_types, lc_list, noshells), 1):
    print(f"The {i}{'st' if i == 1 else 'nd' if i == 2 else 'rd' if i == 3 else 'th'} best fitting structure is {structure_type} with {noshell} shells and lattice constant {lc}.")    
    rwp = generate_and_evaluate_clusters(noshell, lc, structure_type, atom, surfaces, x_target, Int_target, scatteringfunction, plot=True)



Best total structures across all types:
Structure Type: Icosahedron, Details: {'noshell': 7, 'lc': 2.8200000000000003, 'number_of_atoms': 923, 'rwp': 0.015542706935371858}
Structure Type: Icosahedron, Details: {'noshell': 8, 'lc': 2.8200000000000003, 'number_of_atoms': 1415, 'rwp': 0.09674141614834637}
Structure Type: Icosahedron, Details: {'noshell': 6, 'lc': 2.8200000000000003, 'number_of_atoms': 561, 'rwp': 0.1276662682339128}
Structure Type: Icosahedron, Details: {'noshell': 9, 'lc': 2.8200000000000003, 'number_of_atoms': 2057, 'rwp': 0.17531015028760927}
Structure Type: Icosahedron, Details: {'noshell': 6, 'lc': 2.8400000000000003, 'number_of_atoms': 561, 'rwp': 0.20026605112191873}

The 1st best fitting structure is Icosahedron with 7 shells and lattice constant 2.8200000000000003.


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_9095/249943614.py:98: UserWarning: Warning: Your system might have a CUDA-enabled GPU, but CUDA is not available. Computations will run on the CPU instead. For optimal performance, please install Pytorch with CUDA support. If you do not have a CUDA-enabled CPU, you can surpress this warning by specifying the 'device' argument as 'cpu'
  calc = DebyeCalculator(qmin=2, qmax=10.0, rmax=30, qstep=0.01)


The 2nd best fitting structure is Icosahedron with 8 shells and lattice constant 2.8200000000000003.


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_9095/249943614.py:98: UserWarning:




The 3rd best fitting structure is Icosahedron with 6 shells and lattice constant 2.8200000000000003.


The 4th best fitting structure is Icosahedron with 9 shells and lattice constant 2.8200000000000003.


The 5th best fitting structure is Icosahedron with 6 shells and lattice constant 2.8400000000000003.


# Results for simulated Sq data

In [3]:
# Load the results from the brute force search
result_simulated_Sq = load_dict('../results/bruteforce/result_simulated_Sq_fast_StructureSpace.json')

# Load the target PDF data
x_target, Int_target = LoadData(simulated_or_experimental='simulated', scatteringfunction='Sq')

# Get the best structure details
structure_types, lc_list, noshells = get_best_structure_details(result_simulated_Sq, num_best_fits=5, print_best=True)
atom = 'Au'
surfaces = [[1, 0, 0], [1, 1, 0], [1, 1, 1]] # Define the surfaces of the cluster
scatteringfunction = 'Sq'

# Loop through the best structures and calculate the Rwp value for each one
for i, (structure_type, lc, noshell) in enumerate(zip(structure_types, lc_list, noshells), 1):
    print(f"The {i}{'st' if i == 1 else 'nd' if i == 2 else 'rd' if i == 3 else 'th'} best fitting structure is {structure_type} with {noshell} shells and lattice constant {lc}.")    
    rwp = generate_and_evaluate_clusters(noshell, lc, structure_type, atom, surfaces, x_target, Int_target, scatteringfunction, plot=True)



Best total structures across all types:
Structure Type: Icosahedron, Details: {'noshell': 7, 'lc': 2.8200000000000003, 'number_of_atoms': 923, 'rwp': 0.012020302474612329}
Structure Type: Icosahedron, Details: {'noshell': 8, 'lc': 2.8200000000000003, 'number_of_atoms': 1415, 'rwp': 0.12834274589665462}
Structure Type: Icosahedron, Details: {'noshell': 6, 'lc': 2.8200000000000003, 'number_of_atoms': 561, 'rwp': 0.13790971334033902}
Structure Type: Icosahedron, Details: {'noshell': 6, 'lc': 2.8400000000000003, 'number_of_atoms': 561, 'rwp': 0.16149866969148868}
Structure Type: Icosahedron, Details: {'noshell': 7, 'lc': 2.8400000000000003, 'number_of_atoms': 923, 'rwp': 0.16267306102281026}

The 1st best fitting structure is Icosahedron with 7 shells and lattice constant 2.8200000000000003.


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_9095/249943614.py:98: UserWarning:




The 2nd best fitting structure is Icosahedron with 8 shells and lattice constant 2.8200000000000003.


The 3rd best fitting structure is Icosahedron with 6 shells and lattice constant 2.8200000000000003.


The 4th best fitting structure is Icosahedron with 6 shells and lattice constant 2.8400000000000003.


The 5th best fitting structure is Icosahedron with 7 shells and lattice constant 2.8400000000000003.


# Results for experimental Gr data

In [4]:
# Load the results from the brute force search
result_experimental_Gr = load_dict('../results/bruteforce/result_experimental_Gr_fast_StructureSpace.json')

# Load the target PDF data
x_target, Int_target = LoadData(simulated_or_experimental='experimental', scatteringfunction='Gr')

# Get the best structure details
structure_types, lc_list, noshells = get_best_structure_details(result_experimental_Gr, num_best_fits=5, print_best=True)
atom = 'Au'
surfaces = [[1, 0, 0], [1, 1, 0], [1, 1, 1]] # Define the surfaces of the cluster
scatteringfunction = 'Gr'

# Loop through the best structures and calculate the Rwp value for each one
for i, (structure_type, lc, noshell) in enumerate(zip(structure_types, lc_list, noshells), 1):
    print(f"The {i}{'st' if i == 1 else 'nd' if i == 2 else 'rd' if i == 3 else 'th'} best fitting structure is {structure_type} with {noshell} shells and lattice constant {lc}.")    
    rwp = generate_and_evaluate_clusters(noshell, lc, structure_type, atom, surfaces, x_target, Int_target, scatteringfunction, plot=True)



Best total structures across all types:
Structure Type: FaceCenteredCubic, Details: {'noshell': 4, 'lc': 2.8800000000000003, 'number_of_atoms': 128, 'rwp': 0.7776829635777015}
Structure Type: Icosahedron, Details: {'noshell': 2, 'lc': 2.8800000000000003, 'number_of_atoms': 13, 'rwp': 0.7827688988358288}
Structure Type: FaceCenteredCubic, Details: {'noshell': 4, 'lc': 2.9000000000000004, 'number_of_atoms': 128, 'rwp': 0.7834956581165649}
Structure Type: Icosahedron, Details: {'noshell': 2, 'lc': 2.8600000000000003, 'number_of_atoms': 13, 'rwp': 0.7838695242062886}
Structure Type: FaceCenteredCubic, Details: {'noshell': 5, 'lc': 2.8800000000000003, 'number_of_atoms': 250, 'rwp': 0.7844727880723835}

The 1st best fitting structure is FaceCenteredCubic with 4 shells and lattice constant 2.8800000000000003.


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_9095/249943614.py:98: UserWarning:




The 2nd best fitting structure is Icosahedron with 2 shells and lattice constant 2.8800000000000003.


The 3rd best fitting structure is FaceCenteredCubic with 4 shells and lattice constant 2.9000000000000004.


The 4th best fitting structure is Icosahedron with 2 shells and lattice constant 2.8600000000000003.


The 5th best fitting structure is FaceCenteredCubic with 5 shells and lattice constant 2.8800000000000003.


# Results for experimental Sq data

In [5]:
# Load the results from the brute force search
result_experimental_Sq = load_dict('../results/bruteforce/result_experimental_Sq_fast_StructureSpace.json')

# Load the target PDF data
x_target, Int_target = LoadData(simulated_or_experimental='experimental', scatteringfunction='Sq')

# Get the best structure details
structure_types, lc_list, noshells = get_best_structure_details(result_experimental_Sq, num_best_fits=5, print_best=True)
atom = 'Au'
surfaces = [[1, 0, 0], [1, 1, 0], [1, 1, 1]] # Define the surfaces of the cluster
scatteringfunction = 'Sq'

# Loop through the best structures and calculate the Rwp value for each one
for i, (structure_type, lc, noshell) in enumerate(zip(structure_types, lc_list, noshells), 1):
    print(f"The {i}{'st' if i == 1 else 'nd' if i == 2 else 'rd' if i == 3 else 'th'} best fitting structure is {structure_type} with {noshell} shells and lattice constant {lc}.")    
    rwp = generate_and_evaluate_clusters(noshell, lc, structure_type, atom, surfaces, x_target, Int_target, scatteringfunction, plot=True)



Best total structures across all types:
Structure Type: Decahedron, Details: {'noshell': 9, 'lc': 2.8800000000000003, 'number_of_atoms': 25790, 'rwp': 0.8051100687725472}
Structure Type: Decahedron, Details: {'noshell': 9, 'lc': 2.8600000000000003, 'number_of_atoms': 25790, 'rwp': 0.8079175192573451}
Structure Type: Decahedron, Details: {'noshell': 8, 'lc': 2.8800000000000003, 'number_of_atoms': 17931, 'rwp': 0.8321716175847221}
Structure Type: Decahedron, Details: {'noshell': 8, 'lc': 2.8600000000000003, 'number_of_atoms': 17931, 'rwp': 0.8363824810133406}
Structure Type: Decahedron, Details: {'noshell': 7, 'lc': 2.8800000000000003, 'number_of_atoms': 11857, 'rwp': 0.8729443965979347}

The 1st best fitting structure is Decahedron with 9 shells and lattice constant 2.8800000000000003.


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_9095/249943614.py:98: UserWarning:


